In [ ]:
# -----------------------------------------
# GitHub API - Extracción de Repositorios y Commits
# Autor: Gabriel Gonzalez 
# -----------------------------------------

# ========================
# 🔧 1. Importación de librerías
# ========================
import requests  # Para realizar solicitudes HTTP a la API de GitHub
import json      # Para manipular respuestas en formato JSON
import os        # Para verificar si el archivo se guardó

# ========================
# 2. Configuración general de autenticación
# ========================

# URL base para todas las solicitudes a la API de GitHub
BASE_URL = "https://api.github.com"

# Token personal (NO subir tokens reales a repositorios públicos)
GITHUB_TOKEN = ""

# Cabeceras necesarias para autenticar y definir la versión de la API
headers = {
    "Authorization": f"Bearer {GITHUB_TOKEN}",
    "Accept": "application/vnd.github+json",
    "X-GitHub-Api-Version": "2022-11-28"
}

# ========================
# 3. Buscar repositorios por palabra clave
# ========================
query = "machine learning"  # Tema a buscar
params = {
    "q": query,
    "sort": "stars",       # Ordenar por popularidad
    "order": "desc",       # Resultados más populares primero
    "per_page": 5           # Límite de resultados por página
}

# Ejecutamos la solicitud GET al endpoint de búsqueda
try:
    response = requests.get(f"{BASE_URL}/search/repositories", headers=headers, params=params)
    response.raise_for_status()  # Lanza error si el código no es 200
except requests.exceptions.HTTPError as err:
    print("❌ Error HTTP al buscar repositorios:", err)
except requests.exceptions.RequestException as err:
    print("❌ Error general de red:", err)
else:
    data = response.json()
    print(f"\n🔎 Total de resultados encontrados: {data['total_count']}")

    # Mostramos los repositorios encontrados
    for i, repo in enumerate(data['items']):
        print(f"\n{i+1}. {repo['full_name']}")
        print(f"   ⭐ Estrellas: {repo['stargazers_count']}")
        print(f"   📄 Descripción: {repo['description']}")

# ========================
# 4. Función para extraer commits con paginación
# ========================
def get_all_commits(owner, repo, max_pages=3, per_page=5):
    """
    Extrae commits paginados de un repositorio público usando la API de GitHub.

    Parámetros:
        owner (str): Usuario o cuenta dueña del repositorio.
        repo (str): Nombre del repositorio.
        max_pages (int): Cantidad máxima de páginas a recorrer.
        per_page (int): Commits por página (máx. 100).

    Retorna:
        list: Lista de diccionarios con SHA, autor, fecha y mensaje.
    """
    commits_list = []

    for page in range(1, max_pages + 1):
        print(f"\n🔄 Consultando página {page}...")
        url = f"{BASE_URL}/repos/{owner}/{repo}/commits"
        params = {"per_page": per_page, "page": page}

        try:
            response = requests.get(url, headers=headers, params=params)
            response.raise_for_status()
        except requests.exceptions.HTTPError as err:
            print(f"❌ Error HTTP en página {page}:", err)
            break
        except requests.exceptions.RequestException as err:
            print(f"❌ Error de red en página {page}:", err)
            break

        data = response.json()

        if not data:
            print("✅ No hay más commits disponibles.")
            break

        for commit in data:
            commit_info = {
                "sha": commit.get("sha"),
                "author": commit["commit"]["author"].get("name"),
                "date": commit["commit"]["author"].get("date"),
                "message": commit["commit"]["message"]
            }
            commits_list.append(commit_info)

    print(f"\n📦 Total de commits extraídos: {len(commits_list)}")
    return commits_list

# ========================
# 5. Uso de la función con repositorio 'pandas-dev/pandas'
# ========================
owner = "pandas-dev"
repo_name = "pandas"
commits = get_all_commits(owner, repo_name, max_pages=3, per_page=5)

print("\n🧾 Detalle de commits extraídos:")
for i, c in enumerate(commits):
    print(f"\n{i+1}. 📅 {c['date']} | 👤 {c['author']}")
    print(f"   📝 {c['message']}")
    print(f"   🔑 SHA: {c['sha']}")

# ========================
# 6. Guardar commits en archivo JSON
# ========================
output_filename = f"commits_{repo_name}.json"
with open(output_filename, "w", encoding="utf-8") as f:
    json.dump(commits, f, indent=4, ensure_ascii=False)
print(f"\n✅ Archivo guardado exitosamente como '{output_filename}'")

# ========================
# 7. Consultar contenido del repositorio (endpoint /contents)
# ========================
def get_repo_contents(owner, repo, path=""):
    """
    Consulta el contenido de un repositorio (archivos y carpetas) usando /contents

    Parámetros:
        owner (str): Usuario o organización
        repo (str): Nombre del repositorio
        path (str): Ruta del directorio a consultar ("" es raíz)

    Retorna:
        list: Lista de archivos o carpetas encontrados
    """
    url = f"{BASE_URL}/repos/{owner}/{repo}/contents/{path}"
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
    except requests.exceptions.HTTPError as err:
        print("❌ Error HTTP al consultar contenido:", err)
        return []
    except requests.exceptions.RequestException as err:
        print("❌ Error de red al consultar contenido:", err)
        return []

    data = response.json()
    print(f"\n📁 Contenido del repositorio '{repo}' en ruta '{path}':")
    for item in data:
        tipo = "📄 Archivo" if item['type'] == 'file' else "📁 Carpeta"
        print(f"- {tipo}: {item['name']}")
    return data

# Consultamos la raíz del repositorio de pandas
directory_listing = get_repo_contents(owner, repo_name)

# ========================
# 8. Recomendaciones de manejo de errores comunes
# ========================
print("\n📌Recomendaciones de manejo de errores:📌")
print("- 401 Unauthorized → Verifica si el token está vencido o mal escrito.")
print("- 403 Rate Limit → La API de GitHub tiene límites por hora. Intenta luego o autentica correctamente.")
print("- 404 Not Found → Verifica si el repositorio o ruta existe.")
print("- Error de red → Revisa tu conexión o el servidor de GitHub.")